In [57]:
import pandas as pd

In [58]:
df = pd.read_csv(
    './data/src/11111-01-01-4.csv', 
    delimiter=';', 
    encoding='latin1', 
    skiprows=6, 
    skipfooter=4,
    names=(
        'date', 'id', 'name', 'val'
    )
).applymap(lambda x: x.strip())
del df['date']
del df['val']
df = df[df['id'].map(len) == 5]
df.head()

/home/elliot/Projects/datenland/graphql-datastore/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


,id,name
2,01001,"Flensburg, Kreisfreie Stadt"
3,01002,"Kiel, Landeshauptstadt, Kreisfreie Stadt"
4,01003,"Lübeck, Hansestadt, Kreisfreie Stadt"
5,01004,"Neumünster, Kreisfreie Stadt"
6,01051,"Dithmarschen, Landkreis"


In [59]:
def get_name_ext(name):
    try:
        ext = name.split(', ')[1]
        if ext == 'Landkreis':
            return 'Kreis'
        return ext
    except IndexError:
        return 'Kreis'
    
def cleanup_name(name):
    name = name.split(', ')[0]
    for part in ('Kreisfreie Stadt', 'Landkreis'):
        name = name.replace(part, '')
    return name

In [60]:
df['name_ext'] = df['name'].map(get_name_ext)
df['name'] = df['name'].map(cleanup_name)
df.head()

,id,name,name_ext
2,01001,Flensburg,Kreisfreie Stadt
3,01002,Kiel,Landeshauptstadt
4,01003,Lübeck,Hansestadt
5,01004,Neumünster,Kreisfreie Stadt
6,01051,Dithmarschen,Kreis


In [61]:
df.tail()

,id,name,name_ext
521,16073,Saalfeld-Rudolstadt,Kreis
522,16074,Saale-Holzland-Kreis,Kreis
523,16075,Saale-Orla-Kreis,Kreis
524,16076,Greiz,Kreis
525,16077,Altenburger Land,Kreis


## geocode districts center

In [62]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [63]:
loc = geolocator.geocode('Altenburger Land, Landkreis, Germany')
loc

Location(Altenburger Land, Landkreis Altenburger Land, Thüringen, Deutschland, (50.9574967, 12.3084845703079, 0.0))

In [64]:
loc.latitude, loc.longitude

(50.9574967, 12.3084845703079)

In [65]:
str(loc)

'Altenburger Land, Landkreis Altenburger Land, Thüringen, Deutschland'

In [66]:
def geocode_row(row):
    try:
        loc = geolocator.geocode(', '.join((row['name'], 'Germany')), timeout=5)
        if loc:
            print('geocoded: %s' % str(loc))
            return loc.latitude, loc.longitude
        else:
            loc = geolocator.geocode(', '.join((row['name'], row['name_ext'], 'Germany')), timeout=5)
            if loc:
                print('geocoded: %s' % str(loc))
                return loc.latitude, loc.longitude
    except:
        return None

In [67]:
df['loc'] = df.apply(geocode_row, axis=1)
df.head()

geocoded: Flensburg, Schleswig-Holstein, 24937, Deutschland
geocoded: Kiel, Schleswig-Holstein, 24103, Deutschland
geocoded: Lübeck, Schleswig-Holstein, 23552, Deutschland
geocoded: Neumünster, Schleswig-Holstein, 24534, Deutschland
geocoded: Dithmarschen, A 23, Hoye, Schafstedt, Mitteldithmarschen, Kreis Dithmarschen, Schleswig-Holstein, 25725, Deutschland
geocoded: Köthel (Herzogtum Lauenburg), Schwarzenbek-Land, Kreis Herzogtum Lauenburg, Schleswig-Holstein, 22929, Deutschland
geocoded: Nordfriesland, Schleswig-Holstein, Deutschland
geocoded: Ostholstein, Schleswig-Holstein, Deutschland
geocoded: Pinneberg, Kreis Pinneberg, Schleswig-Holstein, 25421, Deutschland
geocoded: Plön, Kreis Plön, Schleswig-Holstein, 24306, Deutschland
geocoded: Rendsburg-Eckernförde, Rader Insel, Schacht-Audorf, Eiderkanal, Kreis Rendsburg-Eckernförde, Schleswig-Holstein, 24794, Deutschland
geocoded: VR Bank Flensburg-Schleswig eG, 9, Friedrich-Ebert-Straße, Rude, Flensburg, Schleswig-Holstein, 24937, Deut

geocoded: Bochum, Regierungsbezirk Arnsberg, Nordrhein-Westfalen, Deutschland
geocoded: Dortmund, Regierungsbezirk Arnsberg, Nordrhein-Westfalen, 44135, Deutschland
geocoded: Hagen, Regierungsbezirk Arnsberg, Nordrhein-Westfalen, 58095, Deutschland
geocoded: Hamm, Regierungsbezirk Arnsberg, Nordrhein-Westfalen, Deutschland
geocoded: Herne, Regierungsbezirk Arnsberg, Nordrhein-Westfalen, 44623, Deutschland
geocoded: Ennepe-Ruhr-Kreis, Regierungsbezirk Arnsberg, Nordrhein-Westfalen, Deutschland
geocoded: Hochsauerlandkreis, Regierungsbezirk Arnsberg, Nordrhein-Westfalen, Deutschland
geocoded: Märkischer Kreis, Regierungsbezirk Arnsberg, Nordrhein-Westfalen, Deutschland
geocoded: Olpe, Kreis Olpe, Regierungsbezirk Arnsberg, Nordrhein-Westfalen, 57462, Deutschland
geocoded: Siegen-Wittgenstein, Unter den Linden, Brauersdorf, Netphen, Kreis Siegen-Wittgenstein, Regierungsbezirk Arnsberg, Nordrhein-Westfalen, 57250, Deutschland
geocoded: Soest, Kreis Soest, Regierungsbezirk Arnsberg, Nordrhe

geocoded: Freiburg im Breisgau, Regierungsbezirk Freiburg, Baden-Württemberg, 79106, Deutschland
geocoded: Breisgau-Hochschwarzwald, Regierungsbezirk Freiburg, Baden-Württemberg, 79289, Deutschland
geocoded: Emmendingen (Kernstadt), Emmendingen, VVG der Stadt Emmendingen, Landkreis Emmendingen, Regierungsbezirk Freiburg, Baden-Württemberg, 79312, Deutschland
geocoded: Ortenaukreis, Regierungsbezirk Freiburg, Baden-Württemberg, Deutschland
geocoded: Rottweil, Verwaltungsgemeinschaft Rottweil, Landkreis Rottweil, Regierungsbezirk Freiburg, Baden-Württemberg, 78628, Deutschland
geocoded: Schwarzwald-Baar-Kreis, Regierungsbezirk Freiburg, Baden-Württemberg, Deutschland
geocoded: Tuttlingen, Verwaltungsgemeinschaft Tuttlingen, Landkreis Tuttlingen, Regierungsbezirk Freiburg, Baden-Württemberg, 78532, Deutschland
geocoded: Konstanz, Verwaltungsgemeinschaft Konstanz, Landkreis Konstanz, Regierungsbezirk Freiburg, Baden-Württemberg, Deutschland
geocoded: Lörrach, Verwaltungsgemeinschaft Lörrac

geocoded: Günzburg, Landkreis Günzburg, Schwaben, Bayern, 89312, Deutschland
geocoded: Neu-Ulm, Landkreis Neu-Ulm, Schwaben, Bayern, Deutschland
geocoded: Lindau (Bodensee), Landkreis Lindau, Schwaben, Bayern, 88131, Deutschland
geocoded: Ostallgäu, Schwaben, Bayern, 87640, Deutschland
geocoded: Unterallgäu, Schwaben, Bayern, 87746, Deutschland
geocoded: Donau-Ries, Schwaben, Bayern, 86685, Deutschland
geocoded: Oberallgäu, Schwaben, Bayern, 87549, Deutschland
geocoded: Saarbrücken, Regionalverband Saarbrücken, Saarland, Deutschland
geocoded: Merzig, Landkreis Merzig-Wadern, Saarland, 66663, Deutschland
geocoded: Neunkirchen, Landkreis Neunkirchen, Saarland, Deutschland
geocoded: Saarlouis, Landkreis Saarlouis, Saarland, 66740, Deutschland
geocoded: Saarpfalz-Kreis, Saarland, Deutschland
geocoded: Sankt Wendel, Landkreis St. Wendel, Saarland, 66606, Deutschland
geocoded: Brandenburg an der Havel, Brandenburg, Deutschland
geocoded: Cottbus, Brandenburg, 03044, Deutschland
geocoded: Fran

geocoded: Stendal, Landkreis Stendal, Sachsen-Anhalt, 39576, Deutschland
geocoded: Wittenberg, Landkreis Wittenberg, Sachsen-Anhalt, Deutschland
geocoded: Dessau, Dessau-Roßlau, Sachsen-Anhalt, 06842, Deutschland
geocoded: Zerbst, Landkreis Anhalt-Bitterfeld, Sachsen-Anhalt, 39261, Deutschland
geocoded: Bernburg, Salzlandkreis, Sachsen-Anhalt, 06406, Deutschland
geocoded: Bitterfeld, Bitterfeld-Wolfen, Landkreis Anhalt-Bitterfeld, Sachsen-Anhalt, 06749, Deutschland
geocoded: Köthen, Landkreis Anhalt-Bitterfeld, Sachsen-Anhalt, 06366, Deutschland
geocoded: Wittenberg, Landkreis Wittenberg, Sachsen-Anhalt, Deutschland
geocoded: Halle (Saale), Sachsen-Anhalt, 06108, Deutschland
geocoded: Burgenlandkreis, Sachsen-Anhalt, Deutschland
geocoded: Mansfelder Land, Sachsen-Anhalt, 06308, Deutschland
geocoded: Merseburg-Querfurt (Saale), Sachsen-Anhalt, Deutschland
geocoded: Industriegebiet Halle-Saalkreis an der A14 Star Park Halle, Reideburg, Halle (Saale), Sachsen-Anhalt, 06116, Deutschland
ge

,id,name,name_ext,loc
2,01001,Flensburg,Kreisfreie Stadt,"(54.7833021, 9.4333264)"
3,01002,Kiel,Landeshauptstadt,"(54.3216753, 10.1371858)"
4,01003,Lübeck,Hansestadt,"(53.866444, 10.684738)"
5,01004,Neumünster,Kreisfreie Stadt,"(54.0715956, 9.9865296)"
6,01051,Dithmarschen,Kreis,"(54.0744326, 9.2881557)"


In [68]:
df[pd.isnull(df['loc'])]

,id,name,name_ext,loc
95,05334,Städteregion Aachen (einschl. Stadt Aachen),Kreis,None
431,14284,Niederschlesischer Oberlausitzkreis,Kreis,None
435,14290,Weißeritzkreis,Kreis,None
442,14383,Muldentalkreis,Kreis,None
496,15362,Ohrekreis,Kreis,None


In [69]:
df['lat'] = df['loc'].map(lambda x: round(x[0], 3) if x else '')
df['lon'] = df['loc'].map(lambda x: round(x[1], 3) if x else '')
#del df['loc']
df.head()

,id,name,name_ext,loc,lat,lon
2,01001,Flensburg,Kreisfreie Stadt,"(54.7833021, 9.4333264)",54.783,9.433
3,01002,Kiel,Landeshauptstadt,"(54.3216753, 10.1371858)",54.322,10.137
4,01003,Lübeck,Hansestadt,"(53.866444, 10.684738)",53.866,10.685
5,01004,Neumünster,Kreisfreie Stadt,"(54.0715956, 9.9865296)",54.072,9.987
6,01051,Dithmarschen,Kreis,"(54.0744326, 9.2881557)",54.074,9.288


In [71]:
del df['loc']

In [72]:
df.to_csv('./data/src/base.csv', index=False, header=False)